# 모델 매개변수 최적화 하기
이제 모델고 데이터가 준비되었으니, 데이터에 매개변수를 최적화하여 모델을 학습하고, 검증하고, 테스트할 차례입니다.
모델을 학습하는 과정은 반복적인 과정을 거칩니다. 각 반복 단계에서 모델은 출력을 추측하고, 추측과 정답 사이의 오류(loss)를 계산하고, 매개변수에 대한 오류의 도함수(derivative)를 수집한 뒤, gradient descent를 사용하여 이 parameter들을 optimize 합니다.


In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root ="./data",
    train = True,
    download = True,
    transform = ToTensor()
)
test_data = datasets.FashionMNIST(
    root ="./data",
    train = False,
    download = True,
    transform = ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size = 64)
test_dataloader = DataLoader(test_data, batch_size = 64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [5]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# 최적화 단계 (Optimization Loop)
하이퍼파라미터를 설정한 뒤에는 최적화 단게를 통해 모델을 학습하고 최적화 할수 있습니다. 최적화 단계의 각 반복(iteration)을 epoch라고 부릅니다.
하나의 에폭은 다음 두 부분으로 구성됩니다.


*   학습 단계(train loop) - 학습용 데이터셋을 반복(iterate)하고 최적의 매개변수로 수렴합니다.
*   검증/테스트 단계(validation/test loop) - 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 반복합니다.

학습 단계(training loop)에서 일어나는 몇 가지 개념들을 간략히 살펴보겠습니디ㅏ.

# 손실 함수

학습용 데이터를 제공하면, 학습되지 않은 신경망은 정답을 제공하지 않을 확률이 높습니다. 손실 함수(loss function)는 획득한 결과와 실제 값 사이의 틀린 정도(degree of dissimilarity)를 측정하며, 학습 중에 이 값을 최소화하려고 합니다. 주어진 데이터 샘플을 입력으로 계산한 예측과 정답(label)을 비교하여 손실(loss)을 계산합니다.

일반적인 손실함수에는 회귀 문제(regression task)에 사용하는 nn.MSELoss(평균 제곱 오차(MSE; Mean Square Error))나 분류(classification)에 사용하는 nn.NLLLoss (음의 로그 우도(Negative Log Likelihood)), 그리고 nn.LogSoftmax와 nn.NLLLoss를 합친 nn.CrossEntropyLoss 등이 있습니다.

# optimizer

최적화는 각 학습 단계에서 모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정입니다. 최적화 알고리즘은 이 과정이 수행되는 방식(여기에서는 확률적 경사하강법(SGD; Stochastic Gradient Descent))을 정의합니다. 모든 최적화 절차(logic)는 optimizer 객체에 캡슐화(encapsulate)됩니다. 여기서는 SGD 옵티마이저를 사용하고 있으며, PyTorch에는 ADAM이나 RMSProp과 같은 다른 종류의 모델과 데이터에서 더 잘 동작하는 다양한 옵티마이저가 있습니다.

학습하려는 모델의 매개변수와 학습률(learning rate) 하이퍼파라미터를 등록하여 옵티마이저를 초기화합니다.

학습 단계(loop)에서는 최적화는 세단계로 이루어집니다.


*   optimizer.zero_grad() 를 호출하여 모델 매개변수의 변화도를 재설정합니다. 기본적으로 변화도는 더해지기 때문에 중복 계산을 막기 위해 반복할 때 마다 명시적으로 0으로 설정합니다.
*   loss.backwards()를 호출하여 예측 손실(prediction loss)를 역전파 합니다. PyTorch는 각 매개변수에 대한 손실의 변화도를 저장합니다.
*   변화도를 계산한 뒤에는 optimizer.step()을 호출하여 역전파 단게에서 수집된 변화도로 매개변수를 조정합니다.



# 전체구현

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch  %100 == 0:
            loss, current = loss.item(), (batch + 1)*len(X)
            print(f"loss: {loss:>7f} [{current:5>d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct =0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.299218 [64/60000]
loss: 2.288791 [6464/60000]
loss: 2.274296 [12864/60000]
loss: 2.273421 [19264/60000]
loss: 2.245553 [25664/60000]
loss: 2.232956 [32064/60000]
loss: 2.228132 [38464/60000]
loss: 2.203117 [44864/60000]
loss: 2.207247 [51264/60000]
loss: 2.168816 [57664/60000]
Test Error: 
 Accuracy: 40.7%, Avg loss: 2.160180 

Epoch 2
-------------------------------
loss: 2.173233 [64/60000]
loss: 2.161397 [6464/60000]
loss: 2.110194 [12864/60000]
loss: 2.121940 [19264/60000]
loss: 2.075234 [25664/60000]
loss: 2.028837 [32064/60000]
loss: 2.042359 [38464/60000]
loss: 1.975331 [44864/60000]
loss: 1.987012 [51264/60000]
loss: 1.905722 [57664/60000]
Test Error: 
 Accuracy: 53.3%, Avg loss: 1.901927 

Epoch 3
-------------------------------
loss: 1.938120 [64/60000]
loss: 1.904198 [6464/60000]
loss: 1.796592 [12864/60000]
loss: 1.829580 [19264/60000]
loss: 1.728788 [25664/60000]
loss: 1.687254 [32064/60000]
loss: 1.698063 [38464/60000]
loss: